In [1]:
import os
import io
import re
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
listings=pd.read_csv('/Users/andreasaenz/data_science /airbnb/listings-2 copy.csv')

In [3]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2060,https://www.airbnb.com/rooms/2060,20200313233810,2020-03-14,Modern NYC,NaN,"Lovely, spacious, sunny 1 BR apartment in 6th ...","Lovely, spacious, sunny 1 BR apartment in 6th ...",none,NaN,...,f,f,flexible,f,f,1,0,1,0,0.01
1,2595,https://www.airbnb.com/rooms/2595,20200313233810,2020-03-14,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2,2,0,0,0.38
2,3831,https://www.airbnb.com/rooms/3831,20200313233810,2020-03-14,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.71
3,5099,https://www.airbnb.com/rooms/5099,20200313233810,2020-03-14,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,none,My neighborhood in Midtown East is called Murr...,...,f,f,moderate,t,t,1,1,0,0,0.59
4,5114,https://www.airbnb.com/rooms/5114,20200313233810,2020-03-14,Super Room in Great area.,NaN,"First, the area is PERFECT if you want to visi...","First, the area is PERFECT if you want to visi...",none,NaN,...,f,f,strict_14_with_grace_period,f,f,3,1,2,0,0.56


In [4]:
#shape
total_row = listings.shape[0]
listings.shape

(50796, 106)

In [5]:
#nan=msno.bar(df,labels=True,sort='descending', figsize=(30,12),fontsize=10)

NameError: name 'msno' is not defined

# Columns Selections
This is using NY data. Looks like inside aribnb data is same format so it can be used for other cities. This notebook is dealing with keeping/removing columns to use for the model.


In [6]:
#to get all columns, set option
pd.set_option('display.max_columns', 107)
#to get text with no truncation
pd.set_option('display.max_colwidth', -1)

# First group of columns to remove
Columns with less than 50% populated

In [7]:
#Create a funciton to check if columns have lots of null (50% or more)
def less_than_50_percent(colname):
    isnull_count = listings[colname].isna().sum()
    if isnull_count/total_row > .5:
        return True


In [8]:
columns = list(listings)
remove_columns_0 = []
for column in columns:
    remove_column_y_n = less_than_50_percent(column)
    if remove_column_y_n:
        remove_columns_0.append(column)

print(remove_columns_0)

['notes', 'thumbnail_url', 'medium_url', 'xl_picture_url', 'square_feet', 'weekly_price', 'monthly_price', 'license', 'jurisdiction_names']


In [9]:
listings.drop(remove_columns_0, inplace=True, axis=1)
#chek if columns are removed
listings.shape

(50796, 97)

# Second group of columns to remove
All values are the same

In [10]:
#check if all records have same value
def all_same_value(colname):
    count_unique = len(listings[colname].unique())
    if count_unique == 1:
        return True

In [11]:
columns = list(listings)
remove_columns_1 = []
for column in columns:
    remove_column_y_n = all_same_value(column)
    if remove_column_y_n:
        remove_columns_1.append(column)

print(remove_columns_1)

['scrape_id', 'experiences_offered', 'country_code', 'country', 'has_availability', 'requires_license', 'is_business_travel_ready']


In [12]:
listings.drop(remove_columns_1, inplace=True, axis=1)
#chek if columns are removed
listings.shape

(50796, 90)

# Third group of columns to remove
All text columns, check by lenght of max characters, excluding amenites column

In [13]:
#get max length
def getmaxlength(colname):
    listings['length'] = listings[colname].str.len()
    sorted = listings['length'].sort_values(ascending = False)
    maxlength = sorted.head(1).values[0]
    listings.drop('length', inplace=True, axis=1)
    return maxlength

In [14]:
#run this for all object type columns and exlcuding amenities column
#if it's more than 300, then remove them
columns = list(listings)
remove_columns_2 = []
for column in columns:
    if listings[column].dtypes == object and column != 'amenities':
        remove_column_y_n = getmaxlength(column)
        if remove_column_y_n > 300:
            remove_columns_2.append(column)

print(remove_columns_2)

['summary', 'space', 'description', 'neighborhood_overview', 'transit', 'access', 'interaction', 'house_rules', 'host_about']


In [15]:
listings.drop(remove_columns_2, inplace=True, axis=1)
#chek if columns are removed
listings.shape

(50796, 81)

# Fourth group of columns to remove
Columns that only have two distinc values and one value counts for 95%

In [16]:
#get ratio for colmuns with only two values
def countratio(colname):
    temp_list = listings.groupby(colname)[colname].count().tolist()
    if len(temp_list) == 2:
        row_count = listings.shape[0]
        lowest_count = min(temp_list)
        ratio = lowest_count/row_count
        return ratio

In [17]:
#run this for all columns
#if it's less than 5%, then remove them
columns = list(listings)
remove_columns_3 = []
for column in columns:
    remove_column_y_n = countratio(column)
    try:
        if remove_column_y_n < .05:
            remove_columns_3.append(column)
    except: ''

print(remove_columns_3)

['last_scraped', 'host_has_profile_pic', 'calendar_last_scraped', 'require_guest_profile_picture', 'require_guest_phone_verification']


In [18]:
listings.drop(remove_columns_3, inplace=True, axis=1)
#chek if columns are removed
listings.shape

(50796, 76)

# Fifth group of columns to remove
This is dealing with pictures and thumbail columns

In [19]:
#create list of picutres and thumbnail columns
remove_columns_4 = []
for col in listings.filter(like='picture').columns:
    remove_columns_4.append(col)

for col in listings.filter(like='thumbnail').columns:
    remove_columns_4.append(col)

remove_columns_4

['picture_url', 'host_picture_url', 'host_thumbnail_url']

In [20]:
listings.drop(remove_columns_4, inplace=True, axis=1)
#chek if columns are removed
listings.shape

(50796, 73)

# Export data to another csv
Filename is selected_columns_listings.csv. To be used by next program

In [22]:
#export columns left to csv to be used with second notebook
path = '../../data/new-york-city-airbnb-open-data/'

listings_csv = os.path.join(path,'selected_columns_listings.csv')
listings.to_csv(listings_csv)